In [4]:
from math import factorial, floor, ceil

def print_table(arr, rows=[], columns=[]):
    if len(rows) > 0:
        for i in range(len(rows)):
            arr[i] = [str(rows[i])] + arr[i]

    if len(columns) > 0:
        arr = [[' '] + list(map(str, columns))] + arr

    row_lengths = [max(map(lambda x: len(str(x)), column)) for column in zip(*arr)]

    for row in arr:
        for i in range(len(row)):
            padding = row_lengths[i] - len(str(row[i])) + 2
            print(f'{" " * floor(padding / 2)}{row[i]}{" " * ceil(padding / 2)}', end='')
        print()

def calculate(lamb, mu, n, mode):
    if mode == 1:
        ro = lamb / mu
        p0 = 1 / (1 + sum((1 / factorial(k)) * ro ** k for k in range(n)))
        p_cancelation = (ro ** n) / factorial(n) * p0
        Q = 1 - p_cancelation
        A = lamb * Q
        n_o = A / mu
        k_load = n_o / n
        k_downtime = 1 - k_load

        return [p_cancelation, Q, A, n_o, k_load, k_downtime]
    elif mode == 2:
        ro = lamb / mu
        p0 = n * mu / (n * mu + lamb)
        p_cancelation = lamb / (n * mu + lamb)
        Q = 1 - p_cancelation
        A = lamb * Q
        n_o = A / mu
        k_load = n_o / n
        k_downtime = 1 - k_load

        return [p_cancelation, Q, A, n_o, k_load, k_downtime]
    else:
        ro = lamb / (n * mu)
        p0 = (1 - ro) / (1 - (ro) ** (n + 1))
        pk = (ro ** n) * p0
        A = lamb * (1 - pk)
        Q = (1 - pk)
        n_o = A / (n * mu)
        k_load = n_o / n
        k_downtime = 1 - k_load

        return [pk, Q, A, n_o, k_load, k_downtime]
    
n = 3
lamb = 4
mu = 2

no_assistance = calculate(lamb, mu, n, mode=1)
unlimited_assistance = calculate(lamb, mu, n, mode=2)
uniform_assistance = calculate(lamb, mu, n, mode=3)

print_table([list(item) for item in zip(*[no_assistance, unlimited_assistance, uniform_assistance][::1])],
            columns=['Без взаимопомощи', 'Неограниченная', 'Равномерная'], 
            rows=['Вероятность отказа', 'Относительная пропускная способность (вероятность обслуживания поступившей заявки)', 
                  'Абсолютая пропускная способность (среднее число заявок, обслуживаемых в единицу времени)', 
                  'Среднее число занятых каналов (в единицу времени)', 'Коэффициент загрузки (в единицу времени)', 'Коэффициент простоя (в единицу времени)'])

                                                                                            Без взаимопомощи     Неограниченная         Равномерная     
                                    Вероятность отказа                                     0.2222222222222222          0.4          0.12307692307692306 
    Относительная пропускная способность (вероятность обслуживания поступившей заявки)     0.7777777777777778          0.6          0.8769230769230769  
 Абсолютая пропускная способность (среднее число заявок, обслуживаемых в единицу времени)  3.111111111111111           2.4          3.5076923076923077  
                    Среднее число занятых каналов (в единицу времени)                      1.5555555555555556          1.2          0.5846153846153846  
                         Коэффициент загрузки (в единицу времени)                          0.5185185185185185  0.39999999999999997  0.19487179487179487 
                         Коэффициент простоя (в единицу времени)                  

In [24]:
import simpy
import numpy as np


class SMO:
    def __init__(self, n, X, Y, mode):  # 1 - без взаимопомощи, 2 - неограниченная взаимопомощь, 3 - равномерная взаимопомощь
        self.mode = mode
        self.env = simpy.Environment()
        self.n = n
        self.X = X
        self.Y = Y
        self.total_arrivals = 0
        self.total_rejections = 0

        if mode == 1:
            self.channels = simpy.Resource(self.env, capacity=n)
        elif mode == 2:
            self.channels = simpy.Resource(self.env, capacity=1)

    def simulate(self, simulation_time):
        self.env.process(self.arrival_process())
        self.env.run(until=simulation_time)
        return self.total_arrivals, self.total_rejections

    def arrival_process(self):
        while True:
            self.total_arrivals += 1
            inter_arrival_time = np.random.exponential((1 / self.X) * 100) / 100.0  # генерация временного интервала между поступлением заявок
            yield self.env.timeout(inter_arrival_time)  # ожидание inter_arrival_time

            with self.channels.request() as request:  # запрос на доступ к ресурсу (каналу обслуживания)
                result = yield request | self.env.timeout(
                    0)  # процесс ожидает либо завершения запроса к ресурсу (request), либо таймаута в 0 времени
                if request in result:
                    self.env.process(self.service_process())  # создание процесса для обслуживания заявки
                else:
                    self.total_rejections += 1

    def service_process(self):
        with self.channels.request() as request:  # запрос на доступ к ресурсу (каналу обслуживания)
            yield request  # процесс приостанавливается, пока не будет удовлетворен запрос к ресурсу (доступ к каналу обслуживания)

            if self.mode == 1:
                service_time = np.random.exponential((1 / self.Y) * 100) / 100.0
            elif self.mode == 2:
                service_time = np.random.exponential((1 / (self.Y * self.n)) * 100) / 100.0

            yield self.env.timeout(service_time)

    def print_result(self):
        if self.mode == 1:
            print('Без взаимопомощи')
        elif self.mode == 2:
            print('Неограниченная взаимопомощь')

        print(f'Число заявок: {self.total_arrivals}')
        print(f'Число отказов: {self.total_rejections}')
        print(f'Вероятность отказа: {self.total_rejections / self.total_arrivals}\n')

n = 3
X = 4  # заявки в минуту
Y = 2  # обслуживание в минуту (1 / 0.5)

no_assistance_smo = SMO(n, X, Y, mode=1)
no_assistance_smo.simulate(1000)
no_assistance_smo.print_result()

unlimited_assistance_smo = SMO(n, X, Y, mode=2)
unlimited_assistance_smo.simulate(1000)
unlimited_assistance_smo.print_result()


Без взаимопомощи
Число заявок: 4015
Число отказов: 855
Вероятность отказа: 0.2129514321295143

Неограниченная взаимопомощь
Число заявок: 3891
Число отказов: 1519
Вероятность отказа: 0.3903880750449756



In [21]:
import numpy as np


class UniformAssistanceSMO:
    def __init__(self, X, Y, n):
        self.X = X
        self.Y = Y * 10
        self.n = n

        self.applications = {}
        self.channels = [0 for _ in range(n)]

        self.total_arrivals = 0
        self.total_rejections = 0

    def _distribute_application(self, application_number, applications_in_progress):
        mid = sum(1 for item in self.channels if item == applications_in_progress[0])
        for i in range(1, len(applications_in_progress)):
            mid += sum(1 for item in self.channels if item == applications_in_progress[i])
        mid //= (len(applications_in_progress) + 1)

        k = 0
        for i in range(self.n):
            if sum(1 for item in self.channels if item == self.channels[i]) > mid:
                self.channels[i] = application_number  # канал переключился на новую заявку
                k += 1
                if k != 0 and k >= mid - 1:
                    break

    def _generate_application(self):
        application_number = max((key for key in self.applications.keys()), default=0) + 1
        workload = np.random.poisson(self.Y)

        if all(channel == 0 for channel in self.channels):
            for i in range(self.n):
                self.channels[i] = application_number  # канал приступил к обработке заявки
        else:
            applications_in_progress = list(set(self.channels))
            if len(applications_in_progress) == self.n:
                self.total_rejections += 1  # заявка получила отказ
            else:
                self._distribute_application(application_number, applications_in_progress)
        self.applications[application_number] = workload

    def simulate(self, time):
        time_passed = 0
        while time_passed < time * 10:
            # Обработка заявок
            if self.channels[0] != 0:
                for channel in self.channels:
                    self.applications[channel] -= 1

                self.channels = [0 if self.applications[channel] <= 0 else channel for channel in
                                 self.channels]  # канал обработал заявку

            uniq = list(set(self.channels))
            if not (len(uniq) == 1 and uniq[0] == 0):
                mid = sum(1 for item in self.channels if item == uniq[0])
                for i in range(1, len(uniq)):
                    mid += sum(1 for item in self.channels if item == uniq[i])
                mid -= sum(1 for item in self.channels if item == 0)
                mid //= (len(uniq))

                for i in range(self.n):
                    if self.channels[i] != 0:
                        continue

                    for j in range(self.n):
                        if self.channels[j] != 0 and (
                                sum(1 for item in self.channels if item == self.channels[i]) >= mid
                                or
                                sum(1 for item in self.channels if item == self.channels[i]) < mid):
                            self.channels[i] = self.channels[j]  # канал завершил обработку и приступил к обработке другой заявки
                            break

            if time_passed % (60 / self.X) == 0:
                self._generate_application()
                self.total_arrivals += 1
            time_passed += 1

    def print_result(self):
        print(f'Число заявок: {self.total_arrivals}')
        print(f'Число отказов: {self.total_rejections}')
        print(f'Вероятность отказа: {self.total_rejections / self.total_arrivals}\n')


# Параметры модели
X = 4  # Интенсивность поступления заявок
n = 3  # Количество каналов
Y = 5

smo = UniformAssistanceSMO(X, Y, n)
smo.simulate(1000)
smo.print_result()

Число заявок: 667
Число отказов: 70
Вероятность отказа: 0.10494752623688156

